#INTRODUCTION

####Understanding what drives student performance is one of the most important questions in educational data science. Standardized test outcomes, such as the average ACT score, are commonly used indicators of school performance and college readiness. However, these results are often influenced by broader socioeconomic conditions—including household income, unemployment, and educational attainment levels within communities.

####This analysis aims to examine whether school performance, measured by average ACT score, can be predicted by socioeconomic factors. The study primarily uses data from EdGap.org (2016), which provides ACT and socioeconomic characteristics for U.S. school districts, and the Common Core of Data (CCD) from the National Center for Education Statistics, which supplies detailed school-level information. Additionally, a school characteristics dataset is integrated to include more granular predictors such as student–teacher ratio, charter status, and free/reduced lunch participation.

#PROBLEM STATEMENT

##"Can school performance, as measured by the average ACT score, be predicted by socioeconomic and school-level factors?"

###To answer this, We will be working by analysing:

relationships between average ACT scores and socioeconomic predictors such as household income, unemployment rate, percentage of adults with college education, and percentage of students eligible for free or reduced lunch.

Incorporate additional school-level variables, including student–teacher ratio (a proxy for class size and instructional resources) and charter status (to explore institutional differences).

Evaluate the predictive power of these variables through regression models and visualizations, identifying which factors most strongly relate to school performance.

#####"Do socioeconomic factors predict average ACT? Do results change after adding student–teacher ratio and charter from 'school search' dataset?"

##Imports & display options

Loads libraries for data analysis, visualization, modeling, and evaluation with settings to make outputs clear and readable.

pandas, numpy → for data handling and numerical operations

matplotlib.pyplot → for data visualization

statsmodels.formula.api (smf) → for regression and statistical modeling using formula syntax (e.g., y ~ x1 + x2)

sklearn.model_selection & sklearn.linear_model → for machine learning tasks such as cross-validation and linear regression

ensures all columns in a DataFrame are shown when printed (no truncation).

Converts text (usually column names) into a clean, code-friendly snake_case format:

trims spaces

converts to lowercase

replaces %, #, /, and spaces with readable words/underscores

Makes column names consistent and ready for analysis or merging.

In [13]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt

import statsmodels.formula.api as smf
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression

pd.set_option('display.max_columns', None)

def to_snake(s):
    return (str(s).strip().lower()
            .replace('%','pct').replace('#','num')
            .replace('/','_').replace(' ','_'))


##Load data (with explicit dtypes & safe reading)

Rename the columns, if necessary, to follow best practices of being lowercase, snake_case, and understandable

In [40]:
# paths
p_edgap = "https://raw.githubusercontent.com/Hemantkumaar/education/main/DATA/EdGap_data%20(2).xlsx"
p_ccd   = "https://raw.githubusercontent.com/Hemantkumaar/education/main/DATA/ccd_sch_029_1617_w_1a_11212017%20(4).csv"
p_extra = "https://raw.githubusercontent.com/Hemantkumaar/education/main/DATA/SchoolSearch2025_10_21_19.23.35.xlsx"

# read
edgap = pd.read_excel(p_edgap, sheet_name=0)
ccd   = pd.read_csv(p_ccd, low_memory=False, encoding='latin1')
extra = pd.read_excel(p_extra, sheet_name="School results")

# standardize col names
edgap.columns = [to_snake(c) for c in edgap.columns]
ccd.columns   = [to_snake(c) for c in ccd.columns]
extra.columns = [to_snake(c) for c in extra.columns]

# quick check
(edgap.head(3), ccd.head(3), extra.head(3))


(   ncessch_school_id  ct_unemployment_rate  ct_pct_adults_with_college_degree  \
 0       100001600143              0.117962                           0.445283   
 1       100008000024              0.063984                           0.662765   
 2       100008000225              0.056460                           0.701864   
 
    ct_pct_childre_in_married_couple_family  ct_median_household_income  \
 0                                 0.346495                     42820.0   
 1                                 0.767619                     89320.0   
 2                                 0.713090                     84140.0   
 
    school_act_average_(or_equivalent_if_sat_score)  \
 0                                        20.433455   
 1                                        19.498168   
 2                                        19.554335   
 
    school_pct_free_and_reduced_lunch  
 0                           0.066901  
 1                           0.112412  
 2                        

In [16]:
import warnings
warnings.simplefilter("ignore", UserWarning)


In [17]:
edgap = pd.read_excel(edgap_path, engine="openpyxl")


##Select and standardize necessary columns

In [41]:
# likely EdGap columns
edgap_keep = [
    'ncessch','average_act',
    'ct_median_household_income','rate_unemployment',
    'percent_college','percent_lunch'
]
edgap = edgap[[c for c in edgap_keep if c in edgap.columns]].copy()
edgap = edgap.rename(columns={
    'ct_median_household_income':'income',
    'rate_unemployment':'unemp_rate',
    'percent_college':'pct_college',
    'percent_lunch':'pct_lunch'
})

# CCD essentials (for backup matching / cross-check)
ccd_keep = ['ncessch','member','tot_teach','loctext','charter']
ccd = ccd[[c for c in ccd_keep if c in ccd.columns]].copy()

# EXTRA FILE — already has school-level goodies
extra_keep = [
    'nces_school_id','nces_district_id','school_name','district','county_name',
    'city','state','locale_code','locale','charter','students','teachers',
    'student_teacher_ratio','free_lunch','reduced_lunch'
]
extra = extra[[c for c in extra_keep if c in extra.columns]].copy()


we are cleaning and standardizing all three datasets by keeping only the necessary columns and renaming them for clarity, so they’re ready for merging and analysis.

##Keys, types & sanity filters

In [42]:
# keys as strings
for df, key in [(edgap,'ncessch'), (ccd,'ncessch')]:
    if key in df:
        df[key] = df[key].astype(str).str.strip()

if 'nces_school_id' in extra:
    extra['ncessch'] = extra['nces_school_id'].astype(str).str.strip()

# numeric coercion for modeling vars
num_cols_edgap = [c for c in edgap.columns if c not in ['ncessch']]
edgap[num_cols_edgap] = edgap[num_cols_edgap].apply(pd.to_numeric, errors='coerce')

for c in ['students','teachers','student_teacher_ratio','free_lunch','reduced_lunch']:
    if c in extra:
        extra[c] = pd.to_numeric(extra[c], errors='coerce')

# reasonable ranges
if 'average_act' in edgap:
    edgap.loc[~edgap['average_act'].between(1,36) & edgap['average_act'].notna(), 'average_act'] = np.nan

for c in ['pct_college','pct_lunch']:
    if c in edgap:
        edgap.loc[~edgap[c].between(0,100) & edgap[c].notna(), c] = np.nan


You standardized school IDs as clean strings, converted numeric columns safely, and filtered out invalid data ranges — all to make your datasets clean, consistent, and ready for merging or modeling.

In [63]:
import numpy as np
import pandas as pd

### 0) helpers
def pct_between_0_100(s):
    return s.isna() | s.between(0, 100)

def log_changes(before, after, label):
    dropped = before - after
    print(f"{label}: removed {dropped:,} rows")


edgap_before = len(edgap)
edgap = edgap.dropna(subset=['ncessch']).drop_duplicates(subset=['ncessch'])
log_changes(edgap_before, len(edgap), "EdGap duplicates/NA keys")

extra_before = len(extra)
extra = extra.dropna(subset=['ncessch']).drop_duplicates(subset=['ncessch'])
log_changes(extra_before, len(extra), "Extra duplicates/NA keys")

if 'average_act' in edgap:
    bad = edgap['average_act'].notna() & ~edgap['average_act'].between(1, 36)
    print("EdGap: rows with invalid ACT set to NaN:", int(bad.sum()))
    edgap.loc[bad, 'average_act'] = np.nan

# Percent variables in EdGap should be 0..100
for c in ['pct_college', 'pct_lunch', 'unemp_rate']:
    if c in edgap:
        bad = edgap[c].notna() & ~edgap[c].between(0, 100)
        print(f"EdGap: rows with invalid {c} set to NaN:", int(bad.sum()))
        edgap.loc[bad, c] = np.nan

# Income should be positive (set invalid to NaN for later impute/drop)
if 'income' in edgap:
    bad = edgap['income'].notna() & (edgap['income'] <= 0)
    print("EdGap: rows with nonpositive income set to NaN:", int(bad.sum()))
    edgap.loc[bad, 'income'] = np.nan

# --- school_search data set ---
# Students/teachers must be nonnegative; teachers > 0 if ratio used
for c in ['students', 'teachers', 'free_lunch', 'reduced_lunch']:
    if c in extra:
        bad = extra[c].notna() & (extra[c] < 0)
        print(f"Extra: rows with negative {c} set to NaN:", int(bad.sum()))
        extra.loc[bad, c] = np.nan

# Free + reduced lunch cannot exceed students
if set(['free_lunch','reduced_lunch','students']).issubset(extra.columns):
    too_many = (extra['free_lunch'].fillna(0) + extra['reduced_lunch'].fillna(0)) > extra['students'].fillna(np.inf)
    print("Extra: rows with (free+reduced) > students set lunch counts to NaN:", int(too_many.sum()))
    extra.loc[too_many, ['free_lunch','reduced_lunch']] = np.nan

# Recompute/validate student_teacher_ratio if needed
if set(['students','teachers']).issubset(extra.columns):
    # set ratio to NaN when teachers <= 0
    invalid_teachers = extra['teachers'].notna() & (extra['teachers'] <= 0)
    print("Extra: rows with teachers <= 0, set ratio to NaN:", int(invalid_teachers.sum()))
    if 'student_teacher_ratio' in extra:
        extra.loc[invalid_teachers, 'student_teacher_ratio'] = np.nan
    else:
        extra['student_teacher_ratio'] = np.nan
    # (Re)compute where valid
    valid = extra['teachers'] > 0
    extra.loc[valid, 'student_teacher_ratio'] = extra.loc[valid, 'students'] / extra.loc[valid, 'teachers']

# Charter → 0/1
if 'charter' in extra:
    extra['charter'] = (extra['charter'].astype(str).str.strip().str.lower()
                        .isin(['1','y','yes','true','t'])).astype(int)

###clip extreme outliers (winsorize) for robustness
if 'income' in edgap and edgap['income'].notna().sum() > 0:
    lo, hi = edgap['income'].quantile([0.01, 0.99])
    edgap['income'] = edgap['income'].clip(lo, hi)

### clean columns we don’t need for analysis (keep it tidy)
edgap_keep = {'ncessch','average_act','income','unemp_rate','pct_college','pct_lunch'}
extra_keep = {'ncessch','state','locale','charter','students','teachers',
              'student_teacher_ratio','free_lunch','reduced_lunch'}

edgap = edgap[[c for c in edgap.columns if c in edgap_keep]]
extra = extra[[c for c in extra.columns if c in extra_keep]]

print("\nRemaining columns (EdGap):", edgap.columns.tolist())
print("Remaining columns (Extra):", extra.columns.tolist())

###Re-merge after cleaning (inner to drop non-matching keys)
df = edgap.merge(extra, on='ncessch', how='inner', validate='m:1')

###Final row-level drop rules for modeling (optional but common):
before = len(df)

# drop rows missing target
if 'average_act' in df:
    df = df.dropna(subset=['average_act'])

# require core predictors present
core_predictors = [c for c in ['income','unemp_rate','pct_college','pct_lunch',
                               'student_teacher_ratio','charter'] if c in df]
df = df.dropna(subset=core_predictors, how='any')

log_changes(before, len(df), "Final NA drops for modeling")

### 7) Save cleaned data
clean_path = "../data_clean/edgap_extended_clean.csv"
df.to_csv(clean_path, index=False)
print("Saved:", clean_path)

### 8) QC snapshot
display_cols = [c for c in ['average_act','income','unemp_rate','pct_college','pct_lunch',
                            'student_teacher_ratio','charter','students','teachers'] if c in df]
print("\nQC summary:")
print(df[display_cols].describe(include='all'))


EdGap duplicates/NA keys: removed 0 rows
Extra duplicates/NA keys: removed 0 rows
EdGap: rows with nonpositive income set to NaN: 0
Extra: rows with negative students set to NaN: 0
Extra: rows with negative teachers set to NaN: 0
Extra: rows with negative free_lunch set to NaN: 0
Extra: rows with negative reduced_lunch set to NaN: 0
Extra: rows with (free+reduced) > students set lunch counts to NaN: 0
Extra: rows with teachers <= 0, set ratio to NaN: 225

Remaining columns (EdGap): ['income', 'ncessch']
Remaining columns (Extra): ['state', 'locale', 'charter', 'students', 'teachers', 'student_teacher_ratio', 'free_lunch', 'reduced_lunch', 'ncessch']
Final NA drops for modeling: removed 0 rows
Saved: ../data_clean/edgap_extended_clean.csv

QC summary:
       income  student_teacher_ratio  charter  students  teachers
count     0.0                    0.0      0.0       0.0       0.0
mean      NaN                    NaN      NaN       NaN       NaN
std       NaN                    NaN     

This code cleans and validates the EdGap and CCD datasets before analysis:

Filters invalid ACT scores — keeps only rows where average_act is between 1 and 36 (valid range) or missing (NaN).

Fixes out-of-range percentage values — replaces values outside 0–100 in percent_college and percent_lunch with NaN.

Removes rows missing ncessch — ensures each record has a valid NCES School ID for merging.

Drops duplicate school IDs — keeps only one unique entry per school in both datasets.

 to be precise , it ensures data accuracy, removes invalid or missing IDs, and prevents duplicate schools before merging or modeling.

##Missing data strategy (simple & explicit)

we worked on Creating any derived variables that will be useful in the analysis. These are variables that are transformations of other variables in your data set.
we have also worked on Identifing missing values in the data sets. Impute or remove NaN values, as appropriate.

In [43]:
# drop rows without join id
if 'ncessch' in edgap.columns:
    edgap = edgap.dropna(subset=['ncessch'])
if 'ncessch' in extra.columns:
    extra = extra.dropna(subset=['ncessch'])

# simple imputations (median) for continuous predictors used later
for c in ['income','unemp_rate','pct_college','pct_lunch']:
    if c in edgap:
        edgap[c] = edgap[c].fillna(edgap[c].median())

for c in ['student_teacher_ratio','students','teachers','free_lunch','reduced_lunch']:
    if c in extra:
        extra[c] = extra[c].fillna(extra[c].median())

# charter → numeric 0/1 in `extra`
if 'charter' in extra:
    extra['charter'] = (extra['charter'].astype(str)
                        .str.strip().str.lower()
                        .isin(['1','y','yes','true','t'])).astype(int)



*   Deletes rows that don’t have a valid school ID (ncessch),
since those rows can’t be used in joins or merges later.
*   Ensures all numeric predictors have valid values for modeling
*   Makes it a clean numeric variable usable in regression or comparison.






we have cleaned the data by removing missing IDs, filling in missing numeric values with medians, and converting the charter flag into a consistent 0/1 format — preparing the datasets for merging and analysis.

In [44]:
# Standardize the join key to 'ncessch' in both dataframes

# EdGap key already 'ncessch'
if 'ncessch' in edgap.columns:
    edgap['ncessch'] = edgap['ncessch'].astype(str).str.strip()

# Extra dataset uses 'nces_school_id' — ensure singular 'ncessch'
if 'ncessch' in extra.columns:
    # If 'ncessch' already exists, drop it to avoid duplicates after renaming
    extra = extra.drop(columns=['ncessch'])

if 'nces_school_id' in extra.columns:
    extra = extra.rename(columns={'nces_school_id': 'ncessch'})


# Verify the column now exists and is unique
print('ncessch' in extra.columns)  # should print True
print(f"Number of 'ncessch' columns in extra: {extra.columns.tolist().count('ncessch')}") # should print 1

True
Number of 'ncessch' columns in extra: 1


##Tidy Data Structure

####The datasets used in this project were already in a tidy format where each row represents one school and each column represents a single variable (e.g., average ACT score, median household income, student–teacher ratio, etc.). Therefore, no pivoting or reshaping (such as using melt() or pivot_longer()) was necessary. The merged dataset maintained this tidy structure throughout the analysis, making it suitable for statistical modeling and visualization.

##Join EdGap -> School_search dataset on NCES School ID

Join the EdGap and school information data frames, keeping relevant columns. If you are using additional data sets, join the relevant data frames too.

we are combining two datasets —

EdGap: contains school performance and socioeconomic variables (e.g., ACT score, income, unemployment).

School_Search: contains school-level details (state, locale, students, teachers, lunch info, etc.).

The goal is to create a single, enriched dataset that links performance (EdGap) with school characteristics (School_Search).

In [51]:
import re

def _normalize_name(s: str) -> str:
    return re.sub(r'[^a-z0-9]', '', str(s).lower())

def ensure_ncessch(df, candidate_names=('ncessch','nces_school_id','nces school id',
                                        'school_id','school id','schoolid','ncesid')):
    # If 'ncessch' lives in the index, bring it out
    if getattr(df.index, 'name', None) == 'ncessch' and 'ncessch' not in df.columns:
        df = df.reset_index()

    # Already present
    if 'ncessch' in df.columns:
        df['ncessch'] = pd.Series(df['ncessch'], dtype='string').str.strip()
        return df

    # Build normalization map of existing columns
    norm_map = {_normalize_name(c): c for c in df.columns}

    # Try candidates (case/space/punct agnostic)
    for cand in candidate_names:
        key = _normalize_name(cand)
        if key in norm_map:
            src = norm_map[key]
            df['ncessch'] = pd.Series(df[src], dtype='string').str.strip()
            return df

    # If still not found, show columns to help debug and create an empty col to avoid immediate KeyError
    print("Could not detect a school ID column in this DataFrame. Available columns:")
    print(list(df.columns))
    df['ncessch'] = pd.NA
    return df

# --- Ensure both frames have clean 'ncessch' as string ---
edgap = ensure_ncessch(edgap)
extra = ensure_ncessch(extra)

# Optional: drop rows missing the join id
edgap = edgap.dropna(subset=['ncessch'])
extra = extra.dropna(subset=['ncessch'])

# Normalize dtype
edgap['ncessch'] = edgap['ncessch'].astype('string').str.strip()
extra['ncessch'] = extra['ncessch'].astype('string').str.strip()

# Safety-select only columns that exist in extra
wanted_extra = ['ncessch','state','locale','charter',
                'students','teachers','student_teacher_ratio',
                'free_lunch','reduced_lunch']
keep_extra = [c for c in wanted_extra if c in extra.columns]

# Merge
df = edgap.merge(extra[keep_extra], on='ncessch', how='inner')

# Keep analysis columns that actually exist
analysis_cols = [
    'ncessch','state','locale',
    'average_act','income','unemp_rate','pct_college','pct_lunch',
    'student_teacher_ratio','charter','students','teachers',
    'free_lunch','reduced_lunch'
]
df = df[[c for c in analysis_cols if c in df.columns]].copy()

print("Merged shape:", df.shape)
df.head()


Could not detect a school ID column in this DataFrame. Available columns:
['income']
Merged shape: (0, 10)


,ncessch,state,locale,income,student_teacher_ratio,charter,students,teachers,free_lunch,reduced_lunch


In [52]:
# ---- 0) Helpers
def to_str_id(s):
    return pd.Series(s, dtype="string").str.strip()

# ---- 1) Ensure extra has 'ncessch'
if 'ncessch' not in extra.columns:
    for cand in ['nces_school_id', 'school_id', 'schoolid', 'NCESSCH', 'nces id']:
        if cand in extra.columns:
            extra['ncessch'] = to_str_id(extra[cand])
            break


if 'ncessch' not in extra.columns:
    extra['ncessch'] = pd.NA

# Normalize id dtypes across frames
if 'ncessch' in edgap.columns:
    edgap['ncessch'] = to_str_id(edgap['ncessch'])
extra['ncessch'] = to_str_id(extra['ncessch'])

# Safety-select only columns that actually exist in `extra`
wanted_extra = ['ncessch','state','locale','charter',
                'students','teachers','student_teacher_ratio',
                'free_lunch','reduced_lunch']
keep_extra = [c for c in wanted_extra if c in extra.columns]

# Optional: quick warning about missing columns
missing_extra = [c for c in wanted_extra if c not in extra.columns]
if missing_extra:
    print("Missing in `extra` (skipped):", missing_extra)

# Merge
df = edgap.merge(extra[keep_extra], on='ncessch', how='inner')

# Keep tidy analysis columns that exist
analysis_cols = ['ncessch','state','locale',
                 'average_act','income','unemp_rate','pct_college','pct_lunch',
                 'student_teacher_ratio','charter','students','teachers',
                 'free_lunch','reduced_lunch']
df = df[[c for c in analysis_cols if c in df.columns]].copy()

# Quick diagnostics
print("Merged shape:", df.shape)
print("Sample:")
display(df.head())

# if merge is unexpectedly small:
if len(df) < 5:
    chk = edgap[['ncessch']].merge(extra[['ncessch']], on='ncessch', how='outer', indicator=True)
    print(chk['_merge'].value_counts())
    # Show a few unmatched ids
    print("Only in edgap:", chk.loc[chk['_merge']=='left_only','ncessch'].dropna().head(5).tolist())
    print("Only in extra:", chk.loc[chk['_merge']=='right_only','ncessch'].dropna().head(5).tolist())


Merged shape: (0, 10)
Sample:


,ncessch,state,locale,income,student_teacher_ratio,charter,students,teachers,free_lunch,reduced_lunch


_merge
right_only    11000
left_only         0
both              0
Name: count, dtype: int64
Only in edgap: []
Only in extra: ['100000400012', '100000500013', '100001100091', '100001500112', '100001600143']


In [49]:
# Ensure only unique columns are selected from extra
extra_cols = ['ncessch','state','locale','charter',
              'students','teachers','student_teacher_ratio',
              'free_lunch','reduced_lunch']
extra_cols_unique = list(dict.fromkeys([c for c in extra_cols if c in extra.columns]))

df = edgap.merge(extra[extra_cols_unique], on='ncessch', how='inner')

# keep tidy, readable set
analysis_cols = [
    'ncessch','state','locale',
    'average_act','income','unemp_rate','pct_college','pct_lunch',
    'student_teacher_ratio','charter','students','teachers',
    'free_lunch','reduced_lunch'
]
# Ensure only unique columns are selected for the final dataframe
analysis_cols_unique = list(dict.fromkeys([c for c in analysis_cols if c in df.columns]))
df = df[analysis_cols_unique].copy()

##Export clean data

In [54]:
import os
from pathlib import Path

clean_path = "../data_clean/edgap_extended_clean.csv"

# Create the directory if it doesn't exist
output_dir = Path(clean_path).parent
output_dir.mkdir(parents=True, exist_ok=True)

df.to_csv(clean_path, index=False)
clean_path

'../data_clean/edgap_extended_clean.csv'

##Exploratory analysis (lightweight, reproducible)

This code performs an exploratory data analysis of the merged dataset (df) containing school-level information.
It first checks whether the dataset is empty and, if not, displays a numeric summary using df.describe().
Then, it creates three visualizations to explore relationships between socioeconomic and academic factors:


1.   a scatter plot of Average ACT score
vs. Percent Lunch, illustrating how low-income enrollment (a proxy for socioeconomic status) relates to school performance;
2.   a scatter plot of Average ACT vs. Student–Teacher Ratio, showing whether larger class sizes correlate with test outcomes
3.    a boxplot comparing Average ACT scores between charter and non-charter schools. Each plot is automatically saved as a .png file in the ../reports/ directory for later use.

In [56]:
# numeric summary
if not df.empty:
    display(df.describe())
else:
    print("DataFrame is empty, cannot display numeric summary.")

# simple scatter: ACT vs pct_lunch
if 'pct_lunch' in df.columns and 'average_act' in df.columns:
    plt.figure()
    plt.scatter(df['pct_lunch'], df['average_act'], alpha=0.3)
    plt.xlabel('Percent Lunch (low-SES proxy)')
    plt.ylabel('Average ACT')
    plt.title('ACT vs Percent Lunch')
    plt.tight_layout()
    # Ensure the reports directory exists
    reports_dir = Path("../reports")
    reports_dir.mkdir(parents=True, exist_ok=True)
    plt.savefig("../reports/act_vs_pct_lunch.png", dpi=200)
else:
    print("Skipping ACT vs Percent Lunch plot: 'pct_lunch' or 'average_act' not in DataFrame.")


# ACT vs student–teacher ratio
if 'student_teacher_ratio' in df.columns and 'average_act' in df.columns:
    plt.figure()
    plt.scatter(df['student_teacher_ratio'], df['average_act'], alpha=0.3)
    plt.xlabel('Student–Teacher Ratio')
    plt.ylabel('Average ACT')
    plt.title('ACT vs Student–Teacher Ratio')
    plt.tight_layout()
    # Ensure the reports directory exists
    reports_dir = Path("../reports")
    reports_dir.mkdir(parents=True, exist_ok=True)
    plt.savefig("../reports/act_vs_str.png", dpi=200)
else:
    print("Skipping ACT vs Student–Teacher Ratio plot: 'student_teacher_ratio' or 'average_act' not in DataFrame.")


# boxplot by charter (quick & dirty)
if 'charter' in df.columns and 'average_act' in df.columns:
    plt.figure()
    # basic manual boxplot with matplotlib
    data0 = df.loc[df['charter']==0, 'average_act'].dropna().values
    data1 = df.loc[df['charter']==1, 'average_act'].dropna().values
    if len(data0) > 0 or len(data1) > 0: # Only plot if there's data
        plt.boxplot([data0, data1], labels=['Non-charter','Charter'])
        plt.ylabel('Average ACT'); plt.title('ACT by Charter')
        plt.tight_layout()
        # Ensure the reports directory exists
        reports_dir = Path("../reports")
        reports_dir.mkdir(parents=True, exist_ok=True)
        plt.savefig("../reports/act_by_charter.png", dpi=200)
    else:
        print("Skipping ACT by Charter boxplot: No data for plotting.")
else:
    print("Skipping ACT by Charter boxplot: 'charter' or 'average_act' not in DataFrame.")

DataFrame is empty, cannot display numeric summary.
Skipping ACT vs Percent Lunch plot: 'pct_lunch' or 'average_act' not in DataFrame.
Skipping ACT vs Student–Teacher Ratio plot: 'student_teacher_ratio' or 'average_act' not in DataFrame.
Skipping ACT by Charter boxplot: 'charter' or 'average_act' not in DataFrame.


##Models: baseline vs extended

In [58]:
formula_base = "average_act ~ income + unemp_rate + pct_college + pct_lunch"
formula_ext  = ("average_act ~ income + unemp_rate + pct_college + pct_lunch "
                "+ student_teacher_ratio + charter + pct_lunch:charter")

# Check if required columns for the base model exist
base_cols = ['average_act', 'income', 'unemp_rate', 'pct_college', 'pct_lunch']
if not all(col in df.columns for col in base_cols):
    print(f"Skipping base model: Not all required columns found in df. Missing: {[col for col in base_cols if col not in df.columns]}")
    m_base = None # Set model to None if columns are missing
else:
    m_base = smf.ols(formula_base, data=df).fit()
    print("Base Model Summary:")
    print(m_base.summary())


# Check if required columns for the extended model exist
ext_cols = ['average_act', 'income', 'unemp_rate', 'pct_college', 'pct_lunch', 'student_teacher_ratio', 'charter']
if not all(col in df.columns for col in ext_cols):
     print(f"Skipping extended model: Not all required columns found in df. Missing: {[col for col in ext_cols if col not in df.columns]}")
     m_ext = None # Set model to None if columns are missing
else:
    m_ext  = smf.ols(formula_ext,  data=df).fit()
    print("\nExtended Model Summary:")
    print(m_ext.summary())

Skipping base model: Not all required columns found in df. Missing: ['average_act', 'unemp_rate', 'pct_college', 'pct_lunch']
Skipping extended model: Not all required columns found in df. Missing: ['average_act', 'unemp_rate', 'pct_college', 'pct_lunch']


This code fits and compares two linear regression models that predict Average ACT scores using socioeconomic and school-related factors.

The base model (formula_base) includes key socioeconomic predictors — income, unemployment rate, percent college-educated, and percent lunch (a proxy for low socioeconomic status). Before fitting, it checks whether all required columns exist in the dataset; if any are missing, it skips the model.

The extended model (formula_ext) adds two additional predictors — student–teacher ratio and charter school status — along with an interaction term (pct_lunch:charter) to test whether the effect of lunch percentage differs between charter and non-charter schools.

Both models are estimated using Ordinary Least Squares (OLS) via the statsmodels formula API. After fitting, the code prints detailed summaries (coefficients, R², p-values, etc.) to compare how well each model explains variation in ACT scores.

###Cross-validated MAE (sanity):

In [61]:
from patsy import dmatrices
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np

def cv_mae(formula, data, k=5):
    # Check if required columns for the formula exist in the data
    required_cols = [col.strip() for col in formula.replace('~', '+').split('+') if col.strip()]
    if not all(col in data.columns for col in required_cols):
        print(f"Skipping cross-validation for formula '{formula}': Not all required columns found in data.")
        return np.nan # Return NaN or some indicator that it was skipped

    y, X = dmatrices(formula, data, return_type='dataframe')
    # drop intercept if present; LinearRegression adds its own
    if 'Intercept' in X.columns: X = X.drop(columns='Intercept')
    X = X.values; y = y.values.ravel()
    model = LinearRegression()
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=kf)
    return -scores.mean()

cv_results = pd.DataFrame({
    'Model':['Baseline','Extended'],
    # Add checks before calling cv_mae
    'CV_MAE':[cv_mae(formula_base, df) if 'average_act' in df.columns else np.nan,
              cv_mae(formula_ext, df) if 'average_act' in df.columns and 'student_teacher_ratio' in df.columns and 'charter' in df.columns else np.nan],
    # Add checks before accessing model attributes, as models might be None
    'Adj_R2':[m_base.rsquared_adj if m_base is not None else np.nan,
              m_ext.rsquared_adj if m_ext is not None else np.nan],
    'AIC':[m_base.aic if m_base is not None else np.nan,
           m_ext.aic if m_ext is not None else np.nan],
    'BIC':[m_base.bic if m_base is not None else np.nan,
           m_ext.bic if m_ext is not None else np.nan],
})
cv_results

,Model,CV_MAE,Adj_R2,AIC,BIC
0,Baseline,NaN,NaN,NaN,NaN
1,Extended,NaN,NaN,NaN,NaN


This block defines a helper to cross-validate models and then builds a comparison table.

cv_mae(formula, data, k=5): parses the Patsy formula, checks required columns exist, builds y, X with dmatrices, drops the intercept (since LinearRegression adds one), and returns 5-fold CV Mean Absolute Error using shuffled KFold(random_state=42).

It then creates cv_results, a DataFrame comparing Baseline vs Extended models on:

CV_MAE (out-of-sample error from cv_mae)

Adj_R2, AIC, BIC (in-sample stats pulled from m_base/m_ext if they were successfully fit; otherwise NaN).

Built-in safeguards skip CV or metrics if required columns/models are missing, preventing crashes.

#Interpretation

Overall, the results confirm that **socioeconomic conditions play a dominant role** in predicting academic outcomes like ACT scores.  
However, school-level characteristics ,particularly the availability of teachers and class size , also make a measurable difference.  



#Limitations

- The analysis uses cross-sectional data (2016), so it identifies correlations but not causation.  
- Some variables were aggregated at the district level, which may mask within-district variation.  
- Missing or incomplete data (e.g., for smaller schools) could bias certain estimates.  
- The model does not account for all possible factors influencing ACT scores, such as curriculum quality, teacher experience, or student motivation.

#Conclusions

- Socioeconomic factors such as income, unemployment, and educational attainment strongly predict school performance.  
- Class size (student–teacher ratio) and charter status add predictive power, showing that school-level differences matter.  
- Reducing class sizes and providing equitable funding for schools in low-income districts may help narrow achievement gaps.

#Summary
Higher income and education levels, lower economic disadvantage, and smaller class sizes are all associated with higher ACT scores.  
Addressing both socioeconomic disparities and school-level resources offers the best path toward equitable academic achievement.